In [72]:
import gymnasium as gym

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from torch.distributions.categorical import Categorical
from tqdm.auto import tqdm
from collections import deque

import numpy as np
import random

import matplotlib.pyplot as plt

In [170]:
env = gym.make("Taxi-v3", render_mode="rgb_array")

In [171]:
def eval_policy(pol):
    rw = []
    ml = []
    for i in range(100):
        obs, info = env.reset()
        r = []
        l = 0
        for _ in range(200):

            action = select_greedy(pol, obs)
            obs, reward, done, truncated, info = env.step(action)

            r.append(reward)
            l += 1
            if done or truncated:
                break

        rw.append(sum(r))
        ml.append(l)

    return sum(rw) / len(rw), sum(ml) / len(ml)

In [179]:
class Agent(nn.Module):
    def __init__(self):
        super(Agent, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(500, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 6),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, x):
        return self.seq(nn.functional.one_hot(x.long(), 500).float().view(-1, 500))


def select_greedy(policy, state):
    with torch.no_grad():
        return torch.argmax(policy(torch.Tensor([state])).exp()).item()

def select(policy, state):
    with torch.no_grad():
        logits = policy(torch.Tensor([state]))
        dist = Categorical(logits=logits)
        return dist.sample().item()



pol = Agent()

optimizer = optim.Adam(pol.parameters(), lr=0.01)
crit = nn.NLLLoss(reduction='none')

for i in tqdm(range(10000)):
    obs, info = env.reset()

    h = []

    for _ in range(200):

        action = select(pol, obs)
        nobs, reward, done, truncated, info = env.step(action)

        h.append((obs, action, reward))

        obs = nobs

        if done or truncated:
            break

    bs, ba, br = zip(*h)

    bs = torch.Tensor(bs)
    ba = torch.LongTensor(ba)

    r = [br[-1]]

    for v in br[::-1][1:]:
        r.append(v + 0.99 * r[-1])

    br = torch.Tensor(r[::-1])
    br = (br - br.mean()) / (br.std() + 1e-8)
    optimizer.zero_grad()
    p = pol(bs.unsqueeze(1))

    loss = torch.sum(crit(p, ba) * br)
    loss.backward()
    optimizer.step()


    if i % 100 == 0:
        print(eval_policy(pol))



  0%|          | 0/10000 [00:00<?, ?it/s]

(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)
(-200.0, 200.0)


KeyboardInterrupt: 